#Ingestion del archivos movie_company csv


### Paso 1 - Leer el archivos multiples csv usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"

In [0]:
%run
"../includes/configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

In [0]:
movie_company_schema = StructType(fields=[
    StructField("movieId", IntegerType(), False),
    StructField("companyId", StringType(), True)
])

In [0]:
movie_company_df = spark.read \
    .schema(movie_company_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/movie_company")


## Parte 2 : Renombrar las columnas y añadir nuevas columnas
##### 1. "personId" renombrar a "person_id"
##### 2. Agregar las columnas "Ingestion_date" y "environment"
##### 3. Agregar la columna "name" a partir de la concatenación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col,concat,current_timestamp,lit



In [0]:
movie_company_final_df = movie_company_df \
                                .withColumnsRenamed({"movieId": "movie_id",
                                                     "companyId": "company_id"}) \
                                .withColumn("ingestion_date", current_timestamp()) \
                                .withColumn("enviroment", lit("Production")) \
                                .withColumn("file_date", lit(v_file_date))





## Parte 3. guardar en formato parquet en el datalake

In [0]:
#overwrite_partition (movie_company_final_df, "movie_silver", "movie_company", "file_date")

In [0]:
#movie_company_final_df.write.mode("overwrite").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movie_company")
merge_key = 'tgt.movie_id = src.movie_id and tgt.company_id = src.company_id'
merge_delta_data(movie_company_final_df,"/mnt/moviehistory7809/silver","movie_silver", "movie_company", "file_date", merge_key)


In [0]:
%sql
select file_date,count(*)
from movie_silver.movie_company
group by file_date



In [0]:
dbutils.notebook.exit("Success No 10")